In [ ]:
from autumn.core.project import get_project
from matplotlib import pyplot
from autumn.core.plots.utils import REF_DATE
# from autumn.calibration.targets import get_target_series
import pandas as pd
from autumn.core.utils.display import pretty_print

from autumn.core.plots.uncertainty.plots import _get_target_values, _plot_targets_to_axis

In [ ]:
project = get_project("sm_sir", "ho_chi_minh_city")

In [ ]:
update_params = {
    "infectious_seed": 5000,
    "contact_rate": 0.2,
    "immunity_stratification.infection_risk_reduction.low": 0.45,
    "testing_to_detection.assumed_tests_parameter": 0.01,
    "testing_to_detection.assumed_cdr_parameter": 0.05,
    "immunity_stratification.prop_immune": 0.01,
    "age_stratification.cfr.multiplier": 1.0,
    "prop_icu_among_hospitalised": 0.05,
    "voc_emergence.omicron.new_voc_seed.start_time": 763.,
    "voc_emergence.omicron.relative_latency": 0.5,
}
params = project.param_set.baseline.update(update_params, calibration_format=True)


In [ ]:
pretty_print(params)

In [ ]:
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [ ]:
outputs = ["notifications", 
           "hospital_occupancy", 
           "infection_deaths",
           "icu_occupancy"] #, "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"]
for output in outputs:
    fig = pyplot.figure(figsize=(12, 8))
    pyplot.style.use("ggplot")
    axis = fig.add_subplot()

    axis = derived_df[output].plot()
    axis.set_title(output)
    
    if output in project.plots:
        values, times = _get_target_values(project.plots, output)        
        date_times = pd.to_datetime(times, origin="01Jan2020",unit="D") 
        _plot_targets_to_axis(axis, values, date_times, on_uncertainty_plot=True)

